In [30]:
import tensorflow as tf
import numpy as np
import inceptionv3
from utils import *
#from defense import *
import os
from wrapper import MyModel
import torch
import time
import pandas as pd
import random

os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [31]:
class blackbox:
    def __init__(self,model):
        self.model = model
        
    def attack_untargeted(self, x0, y0, alpha = 4, beta = 1e-4, iterations = 1000):
        """ Attack the original image and return adversarial example
            model: (pytorch model)
            alpha: learning rate 
            beta: learning rate
            train_dataset: set of training data
            (x0, y0): original image
        """
        pred = self.model.predict(x0)
        print("predicted label:", pred)
        print("true label:", y0)

        if (pred != y0):
            print("Fail to classify the image. No need to attack.")
            return x0,0,0
    
        num_directions = 1000
        num_query = 10
        best_theta, g_theta = None, float('inf')
        query_count = 0
        comp_theta = 0
        current_best = float('inf')
        
        timestart = time.time()
        for i in range(num_directions):
            theta = torch.randn(x0.shape).type(torch.FloatTensor)
            initial_lbd = torch.norm(theta)
            theta = theta/torch.norm(theta)
            if self.model.predict(x0+np.array(initial_lbd*theta)) != y0:
                query_count += 1
                lbd,comp_dec,query = self.fine_grained_binary_search_fix(x0,y0,theta,initial_lbd,g_theta,current_best,num_query)
                query_count += query
                if comp_dec > comp_theta:
                    comp_theta = comp_dec
                    best_theta,g_theta = theta,lbd
                    print("--------> Found abs-distortion %.4f with 10 queries" % g_theta)
                    print("--------> Found comp-distortion %.4f with 10 queries" % comp_dec)
        timeend = time.time()
        print("==========> Found best distortion %.4f in %.4f seconds" % (g_theta, timeend-timestart))
        query_count = (num_directions+1)*num_query
        #print("type of best_theta", type(best_theta))
        #print("type of best_theta", type(g_theta))
        lbd,count = self.fine_grained_binary_search( x0, y0, best_theta, g_theta, current_best)
        g_theta = lbd
        query_count += count
        

        #timestart = time.time()
        print("the best initialization: ",g_theta)
        g1 = 1.0
        theta, g2 = best_theta.clone(), g_theta
        torch.manual_seed(0)
        opt_count = 0
        stopping = 0.01
        prev_obj = 100000
        query_thre = 0
        for i in range(iterations):
            
           # print("iteration:",i)
            if g_theta < 1:
                print("====================query number after distortion < 1 =======================: ",opt_count+query_count)
                query_thre = opt_count+query_count
            gradient = torch.zeros(theta.size())
            q = 10
            min_g1 = float('inf')
            for j in range(q):
                u = torch.randn(theta.size()).type(torch.FloatTensor)
                u = u/torch.norm(u)
                ttt = theta+beta * u
                ttt = ttt/torch.norm(ttt)
                #print("inner loop iteration: ", j)
                g1, count = self.fine_grained_binary_search_local( x0, y0, ttt, initial_lbd = g2, tol=beta/500)
                #print("g1 :",g1)
                opt_count += count
                gradient += (g1-g2)/beta * u
                if g1 < min_g1:
                    min_g1 = g1
                    min_ttt = ttt
            gradient = 1.0/q * gradient
#            print("=============================================")
    
            if (i+1)%50 == 0:
                
                print("Iteration %3d: g(theta + beta*u) = %.4f g(theta) = %.4f distortion %.4f num_queries %d" % (i+1, g1, g2, torch.norm(g2*theta), opt_count))
                if g2 > prev_obj-stopping:
                    break
                prev_obj = g2
    
            min_theta = theta
            min_g2 = g2
            
            #print("gradient:", gradient)
           # print("theta:",theta)
            for _ in range(15):
                new_theta = theta - alpha * gradient
                new_theta = new_theta/torch.norm(new_theta)
                
                new_g2, count = self.fine_grained_binary_search_local( x0, y0, new_theta, initial_lbd = min_g2, tol=beta/50)
                opt_count += count
                alpha = alpha * 2
#                print("alpha in the first for loop is: ",alpha)
                if new_g2 < min_g2:
                    min_theta = new_theta 
                    min_g2 = new_g2
                else:
                    break
#            print("=============================================")
    
            if min_g2 >= g2:
                for _ in range(15):
                    alpha = alpha * 0.25
                    new_theta = theta - alpha * gradient
                    new_theta = new_theta/torch.norm(new_theta)
                    new_g2, count = self.fine_grained_binary_search_local( x0, y0, new_theta, initial_lbd = min_g2, tol=beta/50)
                    opt_count += count
#                    print("alpha in the second for loop is: ",alpha)
                    if new_g2 < g2:
                        min_theta = new_theta 
                        min_g2 = new_g2
                        break
#            print("=============================================")
            if min_g2 <= min_g1:
                theta, g2 = min_theta, min_g2
            else:
                theta, g2 = min_ttt, min_g1
    
            if g2 < g_theta:
                best_theta, g_theta = theta.clone(), g2
            

            if alpha < 1e-6:
                alpha = 1
                print("Warning: not moving, g2 %lf gtheta %lf" % (g2, g_theta))
                beta = beta * 0.1
                if (beta < 1e-10):
                    print("beta is too samll")
                    break
# 
        #print("\nAdversarial Example Found Successfully: distortion %.4f target %d queries %d \nTime: %.4f seconds" % (g_theta, target, query_count + opt_count, timeend-timestart))
        print("inceptionv3_2")
        print("best distortion :", g_theta)
        print("number of queries :", opt_count+query_count)
        return x0 + np.array(g_theta*best_theta), opt_count+query_count, query_thre
   
    def fine_grained_binary_search_fix(self,x0,y0,theta, initial_lbd = 1.0, tol=1e-5,current_best = float('inf'),num_query = 10):
        nquery = 0
        if initial_lbd > current_best: 
            if self.model.predict(x0+ np.array(current_best*theta)) == y0:
                nquery += 1
                return float('inf'), nquery
            lbd = current_best
        else:
            lbd = initial_lbd
    
        lbd_hi = lbd
        lbd_lo = 0.0
    
        while (lbd_hi - lbd_lo) > 1e-5:
            lbd_mid = (lbd_lo + lbd_hi)/2.0
            nquery += 1
            if self.model.predict(x0 + np.array(lbd_mid*theta)) != y0:
                lbd_hi = lbd_mid
            else:
                lbd_lo = lbd_mid
            if nquery > num_query:
                break
        comp_dec = (initial_lbd - lbd_hi)/initial_lbd
       # print("number of query before return for this direction:",nquery)
        return lbd_hi,comp_dec,nquery
    
    
    def fine_grained_binary_search_local(self, x0, y0, theta, initial_lbd = 1.0, tol=1e-5):
        nquery = 0
        lbd = initial_lbd
        
        if self.model.predict(x0+np.array(lbd*theta)) == y0:
            lbd_lo = lbd
            lbd_hi = lbd*1.01
            nquery += 1
            
            while self.model.predict(x0+np.array(lbd_hi*theta)) == y0:
                lbd_hi = lbd_hi*1.01
                nquery += 1
                if lbd_hi > 20:
                    return float('inf'), nquery

        else:
            lbd_hi = lbd
            lbd_lo = lbd*0.99
            nquery += 1
#            timestart2 = time.time()
            while self.model.predict(x0+ np.array(lbd_lo*theta)) != y0 :
                lbd_lo = lbd_lo*0.99
                nquery += 1

        while (lbd_hi - lbd_lo) > tol:
            lbd_mid = (lbd_lo + lbd_hi)/2.0
            nquery += 1
            if self.model.predict(x0 + np.array(lbd_mid*theta)) != y0:
                lbd_hi = lbd_mid
            else:
                lbd_lo = lbd_mid

        return lbd_hi, nquery
    
    
    def fine_grained_binary_search(self, x0, y0, theta, initial_lbd, current_best):
        nquery = 0
        if initial_lbd > current_best: 
            if self.model.predict(x0+ np.array(current_best*theta)) == y0:
                nquery += 1
                #print("initial_lbd > current_best & predict == y0, so return inf")
                return float('inf'), nquery
            lbd = current_best
        else:
            lbd = initial_lbd
            

        lbd_hi = lbd
        lbd_lo = 0
    
        while (lbd_hi - lbd_lo) > 1e-4:
            lbd_mid = (lbd_lo + lbd_hi)/2.0
            nquery += 1
            if self.model.predict(x0 + np.array(lbd_mid*theta)) != y0:
                lbd_hi = lbd_mid
            else:
                lbd_lo = lbd_mid
        #print("find a better initialization")
        return lbd_hi, nquery
    

In [32]:

images,labels = read_images("/data3/ILSVRC2012/train/","/data3/ILSVRC2012/train.txt",20)
model = MyModel(inceptionv3,sess,[0.0,1.0])
attack = blackbox(model)


INFO:tensorflow:Scale of 0 disables regularizer.


In [33]:
compare = []
for i in range(len(images)):
    pred = model.predict(images[i])
    if pred == labels[i]:
        compare.append(1)
    else:
        compare.append(0)
print("accuracy of this model:", sum(compare)/len(compare))

FailedPreconditionError: Attempting to use uninitialized value InceptionV3/Mixed_7c/Branch_0/Conv2d_0a_1x1/weights
	 [[Node: InceptionV3/Mixed_7c/Branch_0/Conv2d_0a_1x1/weights/read = Identity[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](InceptionV3/Mixed_7c/Branch_0/Conv2d_0a_1x1/weights)]]
	 [[Node: ArgMax_8/_7 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1632_ArgMax_8", tensor_type=DT_INT64, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'InceptionV3/Mixed_7c/Branch_0/Conv2d_0a_1x1/weights/read', defined at:
  File "/home/yusu/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/yusu/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/yusu/anaconda3/lib/python3.6/site-packages/ipyker

In [35]:
dist = []
count = []
threshold_query = []
index = [4,5,6,7,8,9,10,11,12,13]
for i in index:
    print("================attacking image ",i+1,"=======================")
    adv,queries,query_thre = attack.attack_untargeted(images[i],labels[i],alpha = 4, beta = 0.05, iterations = 1000)
    dist.append(np.linalg.norm(adv-images[i]))
    count.append(queries)
    threshold_query.append(threshold_query)
    
    

================attacking image  5 =======================


FailedPreconditionError: Attempting to use uninitialized value InceptionV3/Mixed_7c/Branch_0/Conv2d_0a_1x1/weights
	 [[Node: InceptionV3/Mixed_7c/Branch_0/Conv2d_0a_1x1/weights/read = Identity[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](InceptionV3/Mixed_7c/Branch_0/Conv2d_0a_1x1/weights)]]
	 [[Node: ArgMax_8/_7 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1632_ArgMax_8", tensor_type=DT_INT64, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'InceptionV3/Mixed_7c/Branch_0/Conv2d_0a_1x1/weights/read', defined at:
  File "/home/yusu/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/yusu/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/yusu/anaconda3/lib/python3.6/site-packages/ipyker

In [ ]:
index = np.nonzero(count)
index = list(index)[0].tolist()


avg_distortion = np.mean(np.array(dist)[index])
avg_count = np.mean(np.array(count)[index])
avg_thre_query = np.mean(np.array(threshold_query))

print("the average distortion for %2d images :"%(len(index)),avg_distortion)
print("the distortions for 15 images :")
for i in dist:
    print(i)
    
print("the number of queries for %2d images :"%(len(index)), avg_count)    
print("the number of queries for 15 images :")
for j in count:
    print(j)
    
print("the number of queries for %2d images after threshold:"%(len(index)), avg_thre_query)    
print("the number of queries for 15 images :")
for j in threshold_query:
    print(j)



In [5]:
# load images and lables
images,labels = read_images("/data3/ILSVRC2012/train/","/data3/ILSVRC2012/train.txt",10)

In [6]:
model = MyModel(inceptionv3,sess,[0.0,1.0])
attack = blackbox(model)

INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from /home/yusu/obfuscated-gradients/baseline_incepv3/data/inception_v3.ckpt


In [7]:
print(model.predict(images[0]),labels[0])

[449] [449]


In [8]:
compare = []
for i in range(len(images)):
    pred = model.predict(images[i])
    if pred == labels[i]:
        
        compare.append(1)
    else:
        compare.append(0)
print("accuracy of this model:", sum(compare)/len(compare))

accuracy of this model: 0.8


In [ ]:
dist = []
count = []
for i in range(5):
    print("================attacking image ",i+1,"=======================")

    adv,queries = attack.attack_untargeted(images[i],labels[i],alpha = 2, beta = 0.005, iterations = 1000)
    dist.append(np.linalg.norm(adv-images[i]))
    count.append(queries)
    
print("the distortions for 5 images :")
for i in dist:
    print(i)
print("the number of queries for 5 images :")
for j in count:
    print(j)

================attacking image  1 =======================
predicted label: [449]
true label: [449]
--------> Found distortion 160.7198
--------> Found distortion 158.8613
--------> Found distortion 125.9310
--------> Found distortion 107.3789
--------> Found distortion 101.1397
the best initialization:  tensor(101.1397)
Iteration  50: g(theta + beta*u) = 15.0799 g(theta) = 15.0897 distortion 15.0897 num_queries 9753
Iteration 100: g(theta + beta*u) = 8.1369 g(theta) = 8.1328 distortion 8.1328 num_queries 19974
Iteration 150: g(theta + beta*u) = 5.8249 g(theta) = 5.8260 distortion 5.8260 num_queries 29440
Iteration 200: g(theta + beta*u) = 4.8337 g(theta) = 4.8353 distortion 4.8353 num_queries 39316
Iteration 250: g(theta + beta*u) = 3.9063 g(theta) = 3.9060 distortion 3.9060 num_queries 48299
Iteration 300: g(theta + beta*u) = 3.4698 g(theta) = 3.4700 distortion 3.4700 num_queries 56512
Iteration 350: g(theta + beta*u) = 3.1911 g(theta) = 3.1908 distortion 3.1908 num_queries 64798
Ite

In [5]:
print("type of images:",type(images))
print("type of labels: ",type(labels))
print("shape of images", images.get_shape())
print("shape of image:", images[0].get_shape())
print("shape of labels :",labels.get_shape())
print("dims of images:",images.get_shape()[0])

type of images: <class 'tensorflow.python.framework.ops.Tensor'>
type of labels:  <class 'tensorflow.python.framework.ops.Tensor'>
shape of images (100000, 299, 299, 3)
shape of image: (299, 299, 3)
shape of labels : ()
dims of images: 100000


In [ ]:
#sess = tf.InteractiveSession()
#images = images.eval()
#print("type of images",type(images))

In [2]:
all_labels = pd.read_csv("/data3/ILSVRC2012/train.txt",sep=" ",header = None)

In [3]:
print(all_labels[:10])

                                0  1
0  n01440764/n01440764_10026.JPEG  0
1  n01440764/n01440764_10027.JPEG  0
2  n01440764/n01440764_10029.JPEG  0
3  n01440764/n01440764_10040.JPEG  0
4  n01440764/n01440764_10042.JPEG  0
5  n01440764/n01440764_10043.JPEG  0
6  n01440764/n01440764_10048.JPEG  0
7  n01440764/n01440764_10066.JPEG  0
8  n01440764/n01440764_10074.JPEG  0
9   n01440764/n01440764_1009.JPEG  0


In [11]:
all_labels.columns = ['path','label']

In [18]:
new_label = all_labels.label[all_labels.path == 'n04392985/n04392985_10499.JPEG']
new_label = np.asarray(new_label)
if new_label == 848:
    print("bingo!")

bingo!


In [16]:
labels = all_labels[labels.columns[1]][:100000]

In [17]:
index = random.sample(range(0,len(labels)),100)

In [18]:
all_images.get_shape()

TensorShape([Dimension(100000), Dimension(299), Dimension(299), Dimension(3)])

In [19]:
images = [all_images[i] for i in index]

In [20]:
labels = np.asarray(labels[index])

In [21]:
print(len(images))
print(images[0].get_shape())
#print(type(labels))
#print("shape of labels",labels.shape)
print(labels[1])

100
(299, 299, 3)
50


In [ ]:
sess = tf.InteractiveSession()
images = [i.eval() for i in images]
images = images.eval()
print("type of images",type(images))